In [9]:
!pip install evaluate
!pip install nltk
!pip install rouge_score absl-py


  Preparing metadata (setup.py) ... done
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
Using cached absl_py-2.2.2-py3-none-any.whl (135 kB)
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24986 sha256=bb415b1837bd829fe0e5125bec284d0bb66e6646c9cd8b0a64bc056812f19367
  Stored in directory: /home/saitaa0b/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [1]:
import os
import gc
import torch
import pandas as pd
from tqdm import tqdm
from PIL import Image
import glob
from transformers import BlipProcessor, BlipForConditionalGeneration

In [ ]:
# I had to generate this in chunks because the lists couldnt handle all 29000 sentences

df = pd.read_csv("artelingo/dataset/test_arabic/test_arabic.csv", encoding="utf-8-sig")
image_paths = df["image_file"].tolist()

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

model_allam = BlipForConditionalGeneration.from_pretrained(
    os.path.join("blip-finetuned-allam", "checkpoint-31140")
).to("cuda").eval()

model_artelingo = BlipForConditionalGeneration.from_pretrained(
    os.path.join("blip-finetuned-artelingo", "checkpoint-31140")
).to("cuda").eval()

base_path = "../../../../ibex/ai/home/saitaa0b/"
save_every = 5000
os.makedirs("generated_chunks", exist_ok=True)

preds_allam = []
preds_artelingo = []
images_so_far = []
utterances_so_far = []

for idx, path in enumerate(tqdm(image_paths, desc="Generating captions")):
    full_path = os.path.join(base_path, path)
    
    try:
        image = Image.open(full_path).convert("RGB")
    except Exception as e:
        print(f"[Skipped] {full_path} due to error: {e}")
        continue  # dont append anything


    inputs = processor(images=image, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output_allam = model_allam.generate(**inputs, max_length=128)
        caption_allam = processor.tokenizer.decode(output_allam[0], skip_special_tokens=True)

        output_artelingo = model_artelingo.generate(**inputs, max_length=128)
        caption_artelingo = processor.tokenizer.decode(output_artelingo[0], skip_special_tokens=True)

    preds_allam.append(caption_allam)
    preds_artelingo.append(caption_artelingo)
    images_so_far.append(path)
    utterances_so_far.append(df.loc[idx, "utterance_spelled"])

    if (idx + 1) % save_every == 0 or (idx + 1) == len(image_paths):
        df_chunk = pd.DataFrame({
            "image": images_so_far,
            "caption_artelingo": utterances_so_far,
            "pred_allam": preds_allam,
            "pred_artelingo": preds_artelingo
        })
        df_chunk.to_csv(f"generated_chunks/captions_{idx+1}.csv", index=False, encoding="utf-8-sig")
        print(f"Saved chunk at {idx+1} examples")
        
        preds_allam.clear()
        preds_artelingo.clear()
        images_so_far.clear()
        utterances_so_far.clear()
        
        torch.cuda.empty_cache()
        gc.collect()

Generating captions:   7%|▋         | 2041/29061 [56:27<12:41:45,  1.69s/it]

[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Impressionism/joaquã­n-sorolla_bathing.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/saitaa0b/wikiart/Impressionism/joaquã\xadn-sorolla_bathing.jpg'
[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Impressionism/joaquã­n-sorolla_bathing.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/saitaa0b/wikiart/Impressionism/joaquã\xadn-sorolla_bathing.jpg'
[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Impressionism/joaquã­n-sorolla_bathing.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/saitaa0b/wikiart/Impressionism/joaquã\xadn-sorolla_bathing.jpg'
[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Impressionism/joaquã­n-sorolla_bathing.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/saitaa0b/wikiart/Impressionism/joaquã\xadn-sorolla_bathing.jpg'
[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Impressionism/joaquã­n-sorolla_bathing.jpg due to e

Generating captions:   9%|▉         | 2716/29061 [1:15:07<13:05:51,  1.79s/it]

[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Impressionism/joaquã­n-sorolla_fountain-at-the-alcazin-sevilla.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/saitaa0b/wikiart/Impressionism/joaquã\xadn-sorolla_fountain-at-the-alcazin-sevilla.jpg'
[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Impressionism/joaquã­n-sorolla_fountain-at-the-alcazin-sevilla.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/saitaa0b/wikiart/Impressionism/joaquã\xadn-sorolla_fountain-at-the-alcazin-sevilla.jpg'
[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Impressionism/joaquã­n-sorolla_fountain-at-the-alcazin-sevilla.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/saitaa0b/wikiart/Impressionism/joaquã\xadn-sorolla_fountain-at-the-alcazin-sevilla.jpg'
[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Impressionism/joaquã­n-sorolla_fountain-at-the-alcazin-sevilla.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/s

Generating captions:  11%|█         | 3092/29061 [1:25:14<11:37:34,  1.61s/it]

...

Generating captions:  68%|██████▊   | 19888/29061 [9:06:09<5:08:42,  2.02s/it]

[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Cubism/andrã©-lhote_la-plage-1957.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/saitaa0b/wikiart/Cubism/andrã©-lhote_la-plage-1957.jpg'
[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Cubism/andrã©-lhote_la-plage-1957.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/saitaa0b/wikiart/Cubism/andrã©-lhote_la-plage-1957.jpg'
[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Cubism/andrã©-lhote_la-plage-1957.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/saitaa0b/wikiart/Cubism/andrã©-lhote_la-plage-1957.jpg'
[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Cubism/andrã©-lhote_la-plage-1957.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/saitaa0b/wikiart/Cubism/andrã©-lhote_la-plage-1957.jpg'
[Skipped] ../../../../ibex/ai/home/saitaa0b/wikiart/Cubism/andrã©-lhote_la-plage-1957.jpg due to error: [Errno 2] No such file or directory: '/ibex/user/sa

Generating captions:  69%|██████▉   | 20000/29061 [9:09:05<4:20:43,  1.73s/it]

Saved chunk at 20000 examples


Generating captions:  69%|██████▉   | 20033/29061 [9:09:57<4:18:01,  1.71s/it]

In [2]:
csv_files = sorted(glob.glob("generated_chunks/captions_*.csv"))
dfs = [pd.read_csv(f, encoding="utf-8-sig") for f in csv_files]
df_full = pd.concat(dfs, ignore_index=True)
df_full.to_csv("generated_captions.csv", index=False, encoding="utf-8-sig")

print("All chunks merged into 'generated_captions.csv'")

All chunks merged into 'generated_captions.csv'


In [4]:
from evaluate import load
import pandas as pd

df_preds = pd.read_csv("generated_captions.csv", encoding="utf-8-sig")

# extracting predictions and references
refs = df_preds["caption_artelingo"].tolist()  # Ground-truth ArtELingo captions
preds_allam = df_preds["pred_allam"].tolist()  # ALLAM+BLIP predictions
preds_artelingo = df_preds["pred_artelingo"].tolist()  # ArtELingo+BLIP predictions

refs_bleu = [[r] for r in refs]  # BLEU needs list of lists

bleu = load("bleu")
meteor = load("meteor")
rouge = load("rouge")

metrics = {}

metrics["BLEU (ALLAM)"] = bleu.compute(predictions=preds_allam, references=refs_bleu)["bleu"]
metrics["BLEU (ArtELingo)"] = bleu.compute(predictions=preds_artelingo, references=refs_bleu)["bleu"]

metrics["METEOR (ALLAM)"] = meteor.compute(predictions=preds_allam, references=refs)["meteor"]
metrics["METEOR (ArtELingo)"] = meteor.compute(predictions=preds_artelingo, references=refs)["meteor"]

metrics["ROUGE-L (ALLAM)"] = rouge.compute(predictions=preds_allam, references=refs)["rougeL"]
metrics["ROUGE-L (ArtELingo)"] = rouge.compute(predictions=preds_artelingo, references=refs)["rougeL"]

print("\n=== Metric Results ===\n")
for name, value in metrics.items():
    print(f"{name}: {value:.4f}")

metrics_df = pd.DataFrame(list(metrics.items()), columns=["Metric", "Score"])
metrics_df.to_csv("metric_results.csv", index=False, encoding="utf-8-sig")

print("Saved metrics to 'metric_results.csv'")


['جزء من باب من الخشب على شكل مستطيل يبدو كأنه يعبر عن شئ ما', 'برواز حائطي فارغ باللون الفاتح وله اطار بارز بدرجات الوانه', 'رجل و امرأتان في مكان واحد و هم عرايا و ظهور مفاتنهم']


[nltk_data] Downloading package wordnet to /home/saitaa0b/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/saitaa0b/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/saitaa0b/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



=== Metric Results ===

BLEU (ALLAM): 0.0025
BLEU (ArtELingo): 0.0096
METEOR (ALLAM): 0.0385
METEOR (ArtELingo): 0.0494
ROUGE-L (ALLAM): 0.0000
ROUGE-L (ArtELingo): 0.0000
Saved metrics to 'metric_results.csv'
